In [1]:
import pandas as pd
import requests
import time

CSV_FILE = "dataTestEmail.csv"           
TEXT_COLUMN = "testo"                
SUBJECT_COLUMN = "oggetto"           
URL = "https://api-dgswebos4.datagrafservizi.it/sentiment-email"
DELAY = 1                            

pd.set_option('display.max_colwidth', None) 

df = pd.read_csv(CSV_FILE)
results = []

for idx, row in df.iterrows():
    oggetto = str(row[SUBJECT_COLUMN]) if pd.notna(row[SUBJECT_COLUMN]) else ""
    testo = str(row[TEXT_COLUMN]) if pd.notna(row[TEXT_COLUMN]) else ""
    
    full_text = f"Oggetto: {oggetto}. Testo: {testo}"
    payload = {"text": full_text}
    headers = {"Content-Type": "application/json"}

    try:
        response = requests.post(URL, json=payload, headers=headers, timeout=30)
        try:
            resp_json = response.json()
            sentiment = resp_json.get("sentiment", "")
            score = resp_json.get("score", "")
            explanation = resp_json.get("explanation", "")
        except Exception:
            sentiment = ""
            score = ""
            explanation = f"Risposta grezza non JSON: {response.text}"
    except Exception as e:
        sentiment = ""
        score = ""
        explanation = f"Exception richiesta: {e}"

    results.append({
        "input_inviato": full_text,
        "sentiment": sentiment,
        "score": score,
        "explanation": explanation
    })

    time.sleep(DELAY)

df_results = pd.DataFrame(results)
df_results

,input_inviato,sentiment,score,explanation
0,Oggetto: Lancio ufficiale della piattaforma AI. Testo: Il 5 ottobre sarà presentata al pubblico la nuova piattaforma AI con funzionalità di NLP avanzato. È prevista una campagna marketing dedicata.,Positivo,0.95,"Il testo usa parole come 'lancio ufficiale', 'funzionalità di NLP avanzato' e 'campagna marketing dedicata', che sono indicatori di sentiment positivo."
1,Oggetto: Controllo fiscale sui contratti di consulenza. Testo: L’Agenzia delle Entrate ha avviato verifiche su alcune consulenze IT. Serve massima collaborazione con l’ufficio legale.,Neutro,0.60,"Il testo è fornito in un contesto di verifica e collaborazione, suggerendo un approccio generalmente positivo e neutrale. Non vi sono indicazioni di un sentimento molto forte."
2,Oggetto: Obiettivi di delivery raggiunti in anticipo. Testo: Il progetto per il cliente Delta è stato completato con due settimane di anticipo rispetto alla scadenza.,Positivo,0.94,"Il testo esprime soddisfazione e apprezzamento per il completamento del progetto con successo, evidenziando un'anticipazione di due settimane rispetto alla scadenza, il che è un segno positivo."
3,Oggetto: Contestazione legale da parte di un partner tecnologico. Testo: Il partner Omega ci accusa di violazione di accordi di licenza software. È necessario il coinvolgimento immediato del legale.,Negativo,0.85,"Il testo esprime un'accusa e un senso di disagio per una violazione di un accordo, indicando un sentimento negativo."
4,Oggetto: Nota di merito al team cybersecurity. Testo: La direzione ringrazia il reparto sicurezza per aver individuato e bloccato tempestivamente un attacco ransomware.,Positivo,0.94,"Il testo esprime un ringraziamento e una congratulazione per un successo positivo, indicando un sentimento positivo verso il team e la leadership."
5,Oggetto: Feedback positivo sul nuovo helpdesk. Testo: I clienti hanno valutato molto positivamente il portale self-service appena lanciato. Segnalano tempi di risoluzione ridotti del 30%.,Positivo,0.95,Il testo esprime un'elevata soddisfazione e un miglioramento significativo (30%) tramite l'utilizzo del nuovo helpdesk. La frase 'Valutato molto positivamente' suggerisce un sentimento positivo.
6,Oggetto: Problema con la sicurezza di un’app mobile. Testo: È stata rilevata una vulnerabilità critica nella nostra app bancaria. Urge rilascio di una patch per evitare exploit.,Negativo,0.85,"Il testo descrive una vulnerabilità critica, un problema di sicurezza e l'urgenza di un rilascio di patch, indicando un sentimento negativo."
7,Oggetto: Evento aziendale tech day. Testo: La direzione invita tutti i dipendenti al Tech Day annuale con presentazioni su cloud cybersecurity e AI.,Positivo,0.95,"Il testo descrive un evento positivo, incentrato su una forte direzione e una presentazione entusiasmante, suggerendo un sentimento positivo."
8,Oggetto: Segnalazioni su burnout del team sviluppo. Testo: Diversi sviluppatori hanno espresso preoccupazioni sul carico di lavoro eccessivo. Serve valutare un piano di supporto HR.,Negativo,0.60,"Il testo descrive esplicitamente preoccupazioni sul carico di lavoro e un piano di supporto HR, indicando un sentimento negativo."
9,Oggetto: Budget extra per progetti AI approvato. Testo: Il CDA ha approvato un aumento del budget del 20% per le iniziative di intelligenza artificiale.,Positivo,0.95,"Il testo esprime approvazione e aumento di budget, indicando un'ottima valutazione positiva."


In [7]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_DIR = "./sentiment_model"
CSV_FILE = "dataTestEmail.csv"
TEXT_COLUMN = "testo"
SUBJECT_COLUMN = "oggetto"

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
model.eval()

def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    with torch.no_grad():
        outputs = model(**inputs)
        # Regressione → logit unico
        score = outputs.logits.squeeze().item()
    return score

df = pd.read_csv(CSV_FILE)

results = []
for idx, row in df.iterrows():
    oggetto = str(row[SUBJECT_COLUMN]) if pd.notna(row[SUBJECT_COLUMN]) else ""
    testo = str(row[TEXT_COLUMN]) if pd.notna(row[TEXT_COLUMN]) else ""
    
    score_oggetto = predict(oggetto) if oggetto else None
    score_testo = predict(testo) if testo else None
    
    if score_oggetto is not None and score_testo is not None:
        score_finale = (score_oggetto + score_testo) / 2
    else:
        score_finale = score_oggetto or score_testo  
    
    results.append({
        "oggetto": oggetto,
        "testo": testo,
        "score_oggetto": score_oggetto,
        "score_testo": score_testo,
        "score_finale": score_finale
    })

df_results2 = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None) 
df_results2


c:\Users\ruggi\Desktop\finale\.venv-1\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


,oggetto,testo,score_oggetto,score_testo,score_finale
0,Lancio ufficiale della piattaforma AI,Il 5 ottobre sarà presentata al pubblico la nuova piattaforma AI con funzionalità di NLP avanzato. È prevista una campagna marketing dedicata.,0.724623,0.688003,0.706313
1,Controllo fiscale sui contratti di consulenza,L’Agenzia delle Entrate ha avviato verifiche su alcune consulenze IT. Serve massima collaborazione con l’ufficio legale.,0.458798,0.298871,0.378834
2,Obiettivi di delivery raggiunti in anticipo,Il progetto per il cliente Delta è stato completato con due settimane di anticipo rispetto alla scadenza.,0.747275,0.642984,0.695130
3,Contestazione legale da parte di un partner tecnologico,Il partner Omega ci accusa di violazione di accordi di licenza software. È necessario il coinvolgimento immediato del legale.,0.236850,0.120207,0.178528
4,Nota di merito al team cybersecurity,La direzione ringrazia il reparto sicurezza per aver individuato e bloccato tempestivamente un attacco ransomware.,0.739806,0.481020,0.610413
5,Feedback positivo sul nuovo helpdesk,I clienti hanno valutato molto positivamente il portale self-service appena lanciato. Segnalano tempi di risoluzione ridotti del 30%.,0.813035,0.693805,0.753420
6,Problema con la sicurezza di un’app mobile,È stata rilevata una vulnerabilità critica nella nostra app bancaria. Urge rilascio di una patch per evitare exploit.,0.396796,0.246883,0.321840
7,Evento aziendale tech day,La direzione invita tutti i dipendenti al Tech Day annuale con presentazioni su cloud cybersecurity e AI.,0.673007,0.634012,0.653510
8,Segnalazioni su burnout del team sviluppo,Diversi sviluppatori hanno espresso preoccupazioni sul carico di lavoro eccessivo. Serve valutare un piano di supporto HR.,0.227054,0.448214,0.337634
9,Budget extra per progetti AI approvato,Il CDA ha approvato un aumento del budget del 20% per le iniziative di intelligenza artificiale.,0.449170,0.543504,0.496337


In [9]:

def sentiment_regression(score):
    if score < 0.40:
        return "critico"
    elif score < 0.60:
        return "medio"
    else:
        return "positivo"

df_results2['sentiment_regressione'] = df_results2['score_finale'].apply(lambda x: sentiment_regression(x) if pd.notna(x) else None)


df_combined = pd.DataFrame({
    "sentiment llama": df_results['sentiment'],
    "score regressione": df_results2['score_finale'],
    "sentiment regressione": df_results2['sentiment_regressione']
})


pd.set_option('display.max_colwidth', None)
print(df_combined)



   sentiment llama  score regressione sentiment regressione
0         Positivo           0.706313              positivo
1           Neutro           0.378834               critico
2         Positivo           0.695130              positivo
3         Negativo           0.178528               critico
4         Positivo           0.610413              positivo
5         Positivo           0.753420              positivo
6         Negativo           0.321840               critico
7         Positivo           0.653510              positivo
8         Negativo           0.337634               critico
9         Positivo           0.496337                 medio
10        Negativo           0.169815               critico
11        Positivo           0.707988              positivo
12        Negativo           0.356359               critico
13        Positivo           0.812338              positivo
14        Negativo           0.229822               critico
15        Positivo           0.502515   